## Question 14

In [25]:
using LinearAlgebra

function structured_P(L::Int; p::Float64 = 0.45, r::Float64 = 0.01)::Matrix{Float64}
    q = 1 - p - r
    P = diagm(fill(r,L)) + diagm(-1=>fill(q,L-1)) + diagm(1 => fill(p,L-1))
    P[1,1] = 1-p
    P[L,L] = 1-q
    return P
end

structured_π(L::Int; p::Float64 = 0.45, r::Float64 = 0.01)::Vector{Float64} = begin
    q = 1 - p - r
    [(p/q)^i  for i in 1:L] * (q-p) / p / (1-(p/q)^L) #Explicit expression (birth death)
end;

P = structured_P(5)
π = structured_π(5)
@show sum(π)
@show sum(π) ≈ 1
π

sum(π) = 0.9999999999999996
sum(π) ≈ 1 = true


5-element Vector{Float64}:
 0.2786497527413459
 0.2322081272844549
 0.1935067727370457
 0.16125564394753808
 0.13437970328961507

In [ ]:
function get_π_nullspace(P)
    #we wish to solve the equation πP - π = 0
    #i.e. we want the nullspace of (P-I)'
    basis = nullspace((P-I)')
    return basis/sum(basis)
end

get_π_nullspace(structured_P(5))

In [ ]:
function get_π_limit(P,n=100)
    return (P^n)[1,:]
end

get_π_limit(structured_P(5))

In [ ]:
function get_π_eigen(P)
    F = eigen(P')
    π = F.vectors[:,end]
    return π/sum(π)
end

get_π_eigen(structured_P(5))

In [41]:
using StatsBase

function get_π_stats(P,sample_size=10^8)
    n = length(P[1,:])
    π = zeros(n)
    x = 1
    for i in 1:sample_size
        x = wsample(1:n,P[x,:])
        π[x] += 1
    end
    return π/sample_size
end

get_π_stats(structured_P(5))


5-element Vector{Float64}:
 0.27872348
 0.23228161
 0.1935445
 0.16118614
 0.13426427